## Week 3-1 recap

This cell replicates all the code from 3-1. Please refer to the week3-1.ipynb for more info on the code

In [81]:
import bs4 as bs
import urllib.request
import pandas as pd

# requesting data from webpage
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = bs.BeautifulSoup(source,'lxml')

# finding a table
table = soup.find('table')
table_rows = table.find_all('tr')

# fetching table data into layered list
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.split('\n')[0] for tr in td]
    if len(row) > 0 and row[1] != 'Not assigned':
        l.append(row)
        
        
new_list = []
current_postalcode = ''
current_neighborhoods = []
current_borough = ''

# iterating through list, merging duplicate postal codes
for item in l:
    if current_postalcode == item[0]:
        current_neighborhoods = current_neighborhoods + [item[2]]
        if item == l[-1]:
            new_list.append([current_postalcode, current_borough, ', '.join(current_neighborhoods)])
    else:
        if len(current_postalcode) > 0:
            new_list.append([current_postalcode, current_borough, ', '.join(current_neighborhoods)])
        current_postalcode = item[0]
        current_borough = item[1]
        current_neighborhoods = [item[2]]

# reshaping into dataframe
df = pd.DataFrame(data = new_list, columns=['Postal Code','Borough','Neighborhood'])
df.head()

df.shape[0]

103

### installing and importing geocoder

In [66]:
!pip install geocoder
import geocoder # import geocoder


    100% |████████████████████████████████| 102kB 5.9MB/s ta 0:00:01
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: decorator in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ratelim->geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->geocoder)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Pyt

In [72]:
# initialize your variable to None
#lat_lng_coords = None

# loop until you get the coordinates
#while(lat_lng_coords is None):
#  print('ping!')
#  g = geocoder.google('{}, Toronto, Ontario'.format('M5G'))
#  lat_lng_coords = g.latlng  

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]
#print(latitude, longitude)

As stated in course description, geocoder is remarkably unstable. I have tried multiple times but couldn't even get the test data. So we would go the easy way and just copy data from the existing dataset

In [75]:
# fetching geolocation data
geoloc_df = pd.read_csv('https://cocl.us/Geospatial_data')

In [80]:
#viewing the file to be sure
print(geoloc_df.shape[0])
geoloc_df.head()

103


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [82]:
# merging the dataframes by PostalCode

new_df = pd.merge(df, geoloc_df, on='Postal Code')

In [84]:
#checking the newformed dataframe
print(new_df.shape[0])
new_df.head()

103


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Not assigned,43.662301,-79.389494
